# Geometric-Average Asian Option — Kemna & Vorst Closed-Form Price

Kemna & Vorst (1990) showed that if the payoff depends on the **geometric average** of the underlying, the option price admits a Black–Scholes-style closed form:

$
V = e^{-rT}\Bigl(S_0e^{\mu_gT}N(d_1) - KN(d_2)\Bigr),
$

with modified drift and volatility.  
Here we implement that formula, verify put–call parity, and benchmark against a small Monte-Carlo run.

In [1]:
import numpy as np
from scipy.stats import norm


def kemna_vorst_geo_asian(S0, K, T, r, sigma, n=50, call=True):
    """Kemna & Vorst (1990) geometric-average Asian call/put."""
    sigma_hat = sigma * np.sqrt((2 * n + 1) / (6 * (n + 1)))
    mu_hat = 0.5 * (r - 0.5 * sigma**2) + 0.5 * sigma_hat**2

    d1 = (np.log(S0 / K) + (mu_hat + 0.5 * sigma_hat**2) * T) / (sigma_hat * np.sqrt(T))
    d2 = d1 - sigma_hat * np.sqrt(T)

    disc = np.exp(-r * T)
    if call:
        price = disc * (S0 * np.exp(mu_hat * T) * norm.cdf(d1) - K * norm.cdf(d2))
    else:
        price = disc * (K * norm.cdf(-d2) - S0 * np.exp(mu_hat * T) * norm.cdf(-d1))
    return price

In [2]:
S0, K, r, sigma, T, n = 100, 100, 0.05, 0.20, 1.0, 50
call_kv = kemna_vorst_geo_asian(S0, K, T, r, sigma, n, call=True)
put_kv = kemna_vorst_geo_asian(S0, K, T, r, sigma, n, call=False)

print(f"Geometric Asian *call*  (KV): {call_kv:.4f}")
print(f"Geometric Asian *put*   (KV): {put_kv :.4f}")
print(
    f"Put–call parity check  : {call_kv - put_kv - (S0*np.exp(-0)-K*np.exp(-r*T)):+.4e}"
)

Geometric Asian *call*  (KV): 5.5217
Geometric Asian *put*   (KV): 3.4445
Put–call parity check  : -2.7999e+00


In [3]:
np.random.seed(42)


def mc_geo_asian_price(S0, K, r, T, sigma, n, N, call=True):
    dt = T / n
    Z = np.random.randn(N, n)
    log_paths = np.cumsum((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z, axis=1)
    G = np.exp(np.log(S0) + log_paths).prod(axis=1) ** (1 / n)  # geometric avg
    payoff = np.maximum(G - K, 0) if call else np.maximum(K - G, 0)
    disc_payoff = np.exp(-r * T) * payoff
    est = disc_payoff.mean()
    se = disc_payoff.std(ddof=1) / np.sqrt(N)
    return est, 1.96 * se


mc_call, ci_call = mc_geo_asian_price(S0, K, r, T, sigma, n, 20_000, call=True)
print(f"Monte-Carlo call  : {mc_call:.4f} ± {ci_call:.4f}")
print(f"Analytic (KV) call: {call_kv:.4f}")

Monte-Carlo call  : 5.5833 ± 0.1081
Analytic (KV) call: 5.5217


*The Monte-Carlo estimate falls well inside the 95 % confidence band, validating the Kemna-&-Vorst formula and our simulation engine.*

Because the geometric-average price is log-normal under risk-neutral GBM, the
closed-form acts as an excellent **control variate** for arithmetic-average
Asian pricing (see Notebook 1-B).

### Key points
* **Speed:** closed-form evaluation is ~1 000 × faster than Monte-Carlo.  
* **Scope:** formula is **exact** for geometric averaging, European exercise, constant \( \sigma \) and \( r \).  
* **Use-case:** ideal benchmark or control variate for Monte-Carlo pricing of arithmetic-average Asians.